In [17]:
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling  
from transformers import Trainer, TrainingArguments 
import torch  
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer 
import torch.nn.functional as F 
import os
#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
from torch import nn
from math import sqrt

import torch 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline,  AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, Trainer
from transformers import AutoConfig, EarlyStoppingCallback, TrainerCallback
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

2023-07-29 19:07:04.807290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [58]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(0)

# Create the list of pins, users and actions
pins = list(range(0,1000))
users = list(range(1,101))
actions = ['click', 'closeup', 'save']

# Creating empty DataFrame
df = pd.DataFrame(columns=['user', 'pin', 'action', 'timestamp'])

# Populating the DataFrame
for user in users:
    num_pins = np.random.randint(3, 21)  # user engages with min 3 pins and max 20 pins
    engaged_pins = np.random.choice(pins, num_pins, replace=False)  # engaged pins for this user
    engaged_actions = np.random.choice(actions, num_pins)  # actions for this user
    timestamps = [datetime.now() - timedelta(days=x) for x in range(num_pins)]  # random timestamps for user engagement
    
    temp_df = pd.DataFrame({
        'user': user,
        'pin': engaged_pins,
        'action': engaged_actions,
        'timestamp': timestamps
    })
    
    df = pd.concat([df, temp_df])


In [60]:
max(df['pin'])
min(df['pin'])


0

In [2]:
df.head()

,user,pin,action,timestamp
0,1,708,click,2023-07-29 19:04:36.448816
1,1,533,click,2023-07-28 19:04:36.448826
2,1,298,click,2023-07-27 19:04:36.448828
3,1,356,click,2023-07-26 19:04:36.448830
4,1,833,closeup,2023-07-25 19:04:36.448831


In [3]:
#map actions to integers
action_to_int = {'click': 0, 'closeup': 1, 'save': 2}
df['action'] = df['action'].map(action_to_int)


In [4]:
# Split users into train and test users
train_users, test_users = train_test_split(users, test_size=0.2)

train_df = df[df['user'].isin(train_users)]
test_df = df[df['user'].isin(test_users)]


In [5]:
#find the max and min timestamp in train_df
#cutoff_times is 2 weeks prior to the max timestamp
max_timestamp = train_df['timestamp'].max()
min_timestamp = train_df['timestamp'].min()
cut_off_train = max_timestamp - timedelta(days=14)

# Create X_train and y_train
X_train = train_df[train_df['timestamp'] > cut_off_train]
y_train = train_df[(train_df['timestamp'] <= cut_off_train)]

# Create X_test and y_test
X_test = test_df[test_df['timestamp'] > cut_off_train]
y_test = test_df[(test_df['timestamp'] <= cut_off_train)]


In [6]:
y_train.head()

,user,pin,action,timestamp
14,1,862,1,2023-07-15 19:04:36.448845
14,2,664,0,2023-07-15 19:04:36.453075
15,2,780,1,2023-07-14 19:04:36.453077
16,2,352,0,2023-07-13 19:04:36.453078
14,4,271,1,2023-07-15 19:04:36.455612


In [7]:


#if the list is empty add 5 0's or list is less then length 5 add 0 to make it 5 and it if greater then 5 trim it

def pad_or_trim(lst):
    if len(lst) == 0:
        return [0,0,0,0,0]
    elif len(lst) < 5:
        return [0]*(5-len(lst)) + lst
    else:
        return lst[:5]

X_train = X_train.sort_values(by=['timestamp'])
X_train = X_train.groupby('user').agg({'pin': list, 'action': list,'timestamp': list}).reset_index()
X_train['pin'] = X_train['pin'].apply(lambda x: pad_or_trim(x))
X_train['action'] = X_train['action'].apply(pad_or_trim)
X_train['timestamp'] = X_train['timestamp'].apply(pad_or_trim)

y_train = y_train.sort_values(by=['timestamp'])
y_train = y_train.groupby('user').agg({'pin': list, 'action': list,'timestamp': list}).reset_index()
y_train['pin'] = y_train['pin'].apply(pad_or_trim)
y_train['action'] = y_train['action'].apply(pad_or_trim)
y_train['timestamp'] = y_train['timestamp'].apply(pad_or_trim)


X_test = X_test.sort_values(by=['timestamp'])   
X_test = X_test.groupby('user').agg({'pin': list, 'action': list,'timestamp': list}).reset_index()
X_test['pin'] = X_test['pin'].apply(pad_or_trim)
X_test['action'] = X_test['action'].apply(pad_or_trim)
X_test['timestamp'] = X_test['timestamp'].apply(pad_or_trim)

y_test = y_test.sort_values(by=['timestamp'])
y_test = y_test.groupby('user').agg({'pin': list, 'action': list,'timestamp': list}).reset_index()
y_test['pin'] = y_test['pin'].apply(pad_or_trim)
y_test['action'] = y_test['action'].apply(pad_or_trim)
y_test['timestamp'] = y_test['timestamp'].apply(pad_or_trim)



In [8]:
X_train[X_train['user'] == 2]

,user,pin,action,timestamp
1,2,"[531, 163, 687, 778, 18]","[2, 2, 0, 1, 0]","[2023-07-16 19:04:36.453074, 2023-07-17 19:04:..."


In [9]:
y_train[y_train['user'] == 2]

,user,pin,action,timestamp
1,2,"[0, 0, 352, 780, 664]","[0, 0, 0, 1, 0]","[0, 0, 2023-07-13 19:04:36.453078, 2023-07-14 ..."


In [195]:
#DEFINE ALL THE PARAMETERS FOR THE MODEL
hidden_size = 12 #hidden size of transformer and the final output coming from the transformer
pin_embedding_dim = 12 #dimension of the pin embedding
action_embedding_dim = 3 #dimension of the action embedding
num_attention_heads = 12 #number of attention heads in transformer that is concatenated together to form the final attention head of dimension 768
pins_vocab = 1001 #number of pins
actions_vocab = 3 #number of actions
num_layers = 12 #number of transformer layers in the model which is a replication of the same transformer layer whose input is the output of the previous layer and the output is the input to the next layer
dropout = 0.1
max_length = 5 #maximum length of the input sequence
batch_size = 8 #batch size for training
epochs = 1
learning_rate = 0.0001

In [220]:
import torch.nn as nn
class Embeddings(nn.Module):
  """
  Creates a single Dense Embedding for each token --> Token Embedding + Positional Embedding
  """
  def __init__(self, pin_embeddings,actions_vocab, action_embedding_dim, hidden_size):
    super().__init__()
    self.pin_embedding = nn.Embedding.from_pretrained(pin_embeddings)
    self.linear1 = nn.Linear(384, hidden_size)
    self.pin_embedding.weight.requires_grad = True
   
        
    self.action_type_embedding = nn.Embedding(actions_vocab, action_embedding_dim)
    self.linear = nn.Linear(pin_embedding_dim+action_embedding_dim, hidden_size)
    self.position_embedding = nn.Embedding(5, hidden_size)
    self.layer_norm = nn.LayerNorm(hidden_size, eps= 1e-12)
    self.dropout = nn.Dropout()

  def forward(self,pin_ids,action_ids):
    # pin_ids = [batch_size, seq_len]
    # action_ids = [batch_size, seq_len]
    # position_ids = [batch_size, seq_len]
    batch_size, seq_len = pin_ids.shape
    position_ids = torch.arange(seq_len, dtype=torch.long).expand((batch_size, seq_len))
    # position_ids = [batch_size, seq_len]
    pin_embeddings = self.pin_embedding(pin_ids)
    pin_embeddings = self.linear1(pin_embeddings) #linear layer to get the embeddings to the desired hidden size
    # pin_embeddings = [batch_size, seq_len, hidden_size]
    action_embeddings = self.action_type_embedding(action_ids)
    # action_embeddings = [batch_size, seq_len, hidden_size]
    position_embeddings = self.position_embedding(position_ids)
    #print(position_embeddings.shape)

    #concatenate all the pin and action embeddings
    embeddings = torch.cat([pin_embeddings, action_embeddings], dim=-1)
    #apply a linear layer to get the embeddings to the desired hidden size

    embeddings = self.linear(embeddings)
    #print(embeddings.shape)

    #add the position embeddings
    embeddings = embeddings + position_embeddings

    #embeddings = pin_embeddings + action_embeddings + position_embeddings
    # embeddings = [batch_size, seq_len, hidden_size]
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings
    

In [221]:
import pickle
pin_embedding = pickle.load(open("embeddings.pkl", "rb"))

In [222]:
import torch 
position_ids = torch.arange(5, dtype=torch.long).expand((8, 5))
embedded =  nn.Embedding(5, 3)
position_embeddings = embedded(position_ids)
position_embeddings.shape

torch.Size([8, 5, 3])

In [223]:
embedded = Embeddings(pin_embedding,3,3,12)

#sample embeddings

pin_ids = torch.tensor([[1,2,3,4,5],[1,2,3,4,5]])
print(pin_ids.shape)
action_ids = torch.tensor([[1,2,0,2,1],[1,2,0,2,1]])
user_embeddings = embedded(pin_ids,action_ids)
user_embeddings.shape





torch.Size([2, 5])


torch.Size([2, 5, 12])

In [224]:
class pinformer_dataset(Dataset):
  """
  Creates a dataset for the pinformer model
  """
  def __init__(self,df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    pin_ids = torch.tensor(self.df.iloc[idx]['pin'])
    action_ids = torch.tensor(self.df.iloc[idx]['action'])

    target_pin_ids = torch.tensor(self.df.iloc[idx]['pin']) 
    #pick one random non-zero pin from the list of pins and make it the target pin
    target_pin_ids = target_pin_ids[target_pin_ids != 0]
    target_pin_ids = target_pin_ids[torch.randperm(len(target_pin_ids))]
    target_pin_ids = target_pin_ids[0]

    return pin_ids,action_ids, target_pin_ids

train_df = pinformer_dataset(X_train)
test_df = pinformer_dataset(X_test)

train_data_loader = DataLoader(train_df, batch_size=8, shuffle=False) #shuffle is false because we want to preserve the order of the sequence so user 1 embeddings are in sequence and user 2 embeddings are in sequence and so on and can be pulled 
test_data_loader = DataLoader(test_df, batch_size=8, shuffle=False)

In [225]:
from torch import nn
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.head_dim = head_dim #dimension of one head 
    #infeatures=embed_dim
    #outfeatures=head_dim
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)
    
  
  def causal_mask(self,batch_size,size, dtype):  
    mask = torch.tril(torch.ones(size,size)).unsqueeze(0)
    return mask
    
  
      
  def scaled_dot_product_attention(self,query, key, value):
    dim_k = query.size(dim=-1)  
    #print(dim_k)    
    #print(f'Dimension of the q,k,v Matrix [Batch_size, seq_len, Head_dim] of One Head {dim_k}')
    scores = torch.bmm(query,key.transpose(1,2))/ sqrt(dim_k)  #[(1,5,768)*(1,768,5)]/sqrt(768) >>> [batch_size,5,5] 
    
    mask = self.causal_mask(scores.size(0),scores.size(1),dtype=torch.int32)
    #print(mask)
    scores = scores.masked_fill(mask==0, float(0)) 
    weights = F.softmax(scores, dim=-1) #[batch_size,5,5]
    #print(weights)
    #print(f'Softmax for each column across one row {weights}')
    weights_dot_values = torch.bmm(weights,value)  #[batch_size,5,5]*[batch_size,5,64] >>> [batch_size,5,64]
    #print(f'Last Step is to multiply weights and values {weights_dot_values.shape}')
    return weights_dot_values 

  def forward(self, hidden_state):
    #head_state = [batch_size, seq_len, embed_dim]
    #print(f'Input Embedding for Each Token with X Matrix {hidden_state.size()}')
    #q = X*W_q
    q = self.q(hidden_state) #q = [batch_size, seq_len, head_dim]
    #print(f'Shape of the Query Matrix W_q {q.size()}')
    k = self.k(hidden_state) #k = [batch_size, seq_len, head_dim]
    #print(f'Shape of the Key Matrix W_k {k.size()}')
    v = self.k(hidden_state) #v = [batch_size, seq_len, head_dim]
    #print(f'Shape of the Value Matrix W_k {v.size()}')
    #print('-----------------Calculating Self Attention--------------------')
    attn_outputs = self.scaled_dot_product_attention(q,k,v) #attn_outputs = [batch_size, seq_len, head_dim]
    #print(f'Shape of the attention Output with one Head and Head Dimension {self.head_dim} is {attn_outputs.size()}')
    return attn_outputs

In [226]:
#create a single attention head
attention_head = AttentionHead(hidden_size,12)

#sample input first calculate the embeddings for the input
pin_ids = torch.tensor([[1,2,3,4,5],[1,2,3,4,5]])
print(pin_ids.shape)
action_ids = torch.tensor([[1,2,0,2,1],[1,2,0,2,1]])
embedded(pin_ids,action_ids)

#pass the embeddings to the attention head
attention_head(embedded(pin_ids,action_ids))


torch.Size([2, 5])


tensor([[[-0.0239, -0.4851, -0.0187,  0.6908, -0.4658,  0.4746,  0.1577,
           0.5094, -0.0035, -0.1617,  0.1693, -0.3923],
         [ 0.1838, -0.7917,  0.1611,  0.5430,  0.1771,  0.7911, -0.3787,
           0.6660, -0.0925, -0.6892,  0.6260,  0.0477],
         [ 0.0618, -0.5948,  0.0668,  0.4992, -0.1073,  0.5267, -0.0878,
           0.4542, -0.0959, -0.3523,  0.3172, -0.1377],
         [-0.0432, -0.4621, -0.0591,  0.7151, -0.6762,  0.3814,  0.3483,
           0.4264, -0.0356, -0.0203,  0.0213, -0.4844],
         [ 0.0149, -0.5436,  0.0121,  0.6700, -0.3609,  0.5335,  0.0693,
           0.5398, -0.0206, -0.2555,  0.2491, -0.3177]],

        [[-0.2882, -0.1613, -0.1095,  0.0969, -0.4174,  0.1628,  0.5075,
           0.0136, -0.3349,  0.0481, -0.1548, -0.0401],
         [-0.2584, -0.2051, -0.0520,  0.0963, -0.3581,  0.2062,  0.4672,
          -0.0204, -0.3337,  0.0266, -0.1997,  0.0265],
         [-0.3277, -0.1233, -0.1639,  0.1292, -0.5274,  0.1351,  0.5573,
           0.0520, -0.

In [203]:
class MultiHeadAttention(nn.Module):
  def __init__(self, hidden_size, num_attention_heads):
    super().__init__()
    embed_dim = hidden_size
    num_heads = num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)] #initializing all the heads
    self.w_0 = nn.Linear(embed_dim,embed_dim) #the purpose of this linear layer is to concatenate all the heads together to form the final output of the multihead attention

  def forward(self,hidden_state):
    '''
    hidden_state: Input Embedding with dimensions [batch_size, seq_len, embedding_dimension]
    '''
    attention_outputs = [head(hidden_state) for head in self.heads] #Calculating Self-Attention on each head
    contcat_attn_outputs_allheads = torch.cat(attention_outputs, dim=-1) #[batch_size,seq_len, embed_dim]
    Z =   self.w_0(contcat_attn_outputs_allheads) #[batch_size, seq_len, embed_dim]
    return Z

In [204]:
class FeedForward(nn.Module):
  def __init__(self,hidden_size):
    super().__init__()
    self.linear1 = nn.Linear(hidden_size, 3072)
    self.linear2 = nn.Linear(3072, hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(0.1)
  
  def forward(self, attention_outputs):
    output_l1 = self.linear1(attention_outputs)
    activated_outputs = self.gelu(output_l1)
    output_l2 = self.linear2(activated_outputs)
    output = self.dropout(output_l2)
    return output

In [205]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, hidden_size, num_attention_heads):
    super(TransformerDecoderLayer,self).__init__()
    self.layer_norm1 = nn.LayerNorm(hidden_size)
    self.layer_norm2 = nn.LayerNorm(hidden_size)
    self.multi_attention = MultiHeadAttention(hidden_size, num_attention_heads)
    self.feedforward = FeedForward(hidden_size)

  def forward(self, input_embeddings):
     #pre-layer normalization approach
     
     #Step 1: Applying Layer Normalization to Input Embeddings
     normalized_input_embeddings = self.layer_norm1(input_embeddings)
     
     #Step 2: Applying MultiHeadAttention to Normalized Output
     multi_head_attn = self.multi_attention(normalized_input_embeddings)
     
     #Step 3: Add input embeddings to the Multihead Attention Output
     skip_connection_1 = input_embeddings + multi_head_attn

     #step 4: Pass the output to another Layer Normalization 
     layer_norm_2 = self.layer_norm2(skip_connection_1)

     #Step 5: Adding skip connection 1 outputs to the output of the FeedForward Network (applied on Step 4)
     skip_connection_2 = skip_connection_1 + self.feedforward(layer_norm_2)
     #print(f'output of MultiHeadAttention and FeedForward Network is {skip_connection_2.shape}')
     return skip_connection_2

In [227]:
class TransferDecoder(nn.Module):
  def __init__(self,num_attention_heads,num_layers, pin_embedding,actions_vocab,action_embedding_dim, hidden_size):
    super().__init__()
    self.embedding = Embeddings(pin_embedding,actions_vocab,action_embedding_dim, hidden_size)
    self.layers = nn.ModuleList([TransformerDecoderLayer(hidden_size, num_attention_heads) for _ in range(num_layers)]) 
                                
  def forward(self, pin_ids, action_ids):
    embeddings = self.embedding(pin_ids, action_ids) #in: [batch_size, seq_len] out: [batch_size, seq_len, hidden_size]
    for layer in self.layers:
      embeddings = layer(embeddings)
    return embeddings


In [228]:
X_train.iloc[0]['pin']

[239, 985, 999, 674, 994]

In [229]:
#embedding_decoder = TransferDecoder(num_attention_heads,num_layers, pins_vocab,actions_vocab, hidden_size)
embedding_decoder = TransferDecoder(num_attention_heads,num_layers, pin_embedding,actions_vocab,action_embedding_dim, hidden_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#need to pull embeddings for each user and then store it in a dictionary
user1_pins = torch.tensor(X_train.iloc[0]['pin']).unsqueeze(0) #unserequeeze is used to add a dimension to the tensor for batch size dimension is [1,5] for batch size 1 and sequence length 5
user1_actions = torch.tensor(X_train.iloc[0]['action']).unsqueeze(0) #unserequeeze is used to add a dimension to the tensor for batch size dimension is [1,5] for batch size 1 and sequence length 5





In [230]:
user1_embeddings = embedding_decoder(user1_pins,user1_actions)
user1_embeddings.shape

torch.Size([1, 5, 12])

In [231]:
#get embeddings for the first batch
for batch in train_data_loader:
    pin_ids,action_ids, target_pin_ids = batch
    #print(pin_ids.shape)
    #print(action_ids.shape)
    #print(target_pin_ids.shape)
    #print(target_pin_ids)
    #embedding_decoder(pin_ids,action_ids)
    break
  

In [211]:
df.head()

,user,pin,action,timestamp
0,1,708,click,2023-07-29 23:07:29.687074
1,1,533,click,2023-07-28 23:07:29.687086
2,1,298,click,2023-07-27 23:07:29.687089
3,1,356,click,2023-07-26 23:07:29.687091
4,1,833,closeup,2023-07-25 23:07:29.687093


In [246]:
class RecommenderDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx]['user']
        pin_ids = torch.tensor(self.df.iloc[idx]['pin_train'])
        action_ids = torch.tensor(self.df.iloc[idx]['action_train'])
        target_pin_ids = torch.tensor(self.df.iloc[idx]['pin_target'])
        target_action_ids = torch.tensor(self.df.iloc[idx]['action_target'])
        return {'user': user, 'pin_ids': pin_ids, 'action_ids': action_ids, 'target_pin_ids': target_pin_ids, 'target_action_ids': target_action_ids}
       
      


In [247]:
X_train.head()

,user,pin,action,timestamp
0,1,"[239, 985, 999, 674, 994]","[1, 0, 1, 2, 2]","[2023-07-16 19:04:36.448844, 2023-07-17 19:04:..."
1,2,"[531, 163, 687, 778, 18]","[2, 2, 0, 1, 0]","[2023-07-16 19:04:36.453074, 2023-07-17 19:04:..."
2,3,"[993, 179, 297, 760, 124]","[1, 0, 2, 0, 0]","[2023-07-20 19:04:36.454374, 2023-07-21 19:04:..."
3,4,"[630, 738, 798, 344, 885]","[2, 0, 2, 1, 2]","[2023-07-16 19:04:36.455611, 2023-07-17 19:04:..."
4,5,"[439, 332, 36, 380, 460]","[0, 2, 1, 0, 1]","[2023-07-21 19:04:36.456907, 2023-07-22 19:04:..."


In [248]:
merged_train = X_train.merge(y_train, on='user', how='inner')
merged_test = X_test.merge(y_test, on='user', how='inner')

In [249]:
merged_train.rename(columns={'pin_x': 'pin_train', 'action_x': 'action_train', 'timestamp_x': 'timestamp_train', 'pin_y':'pin_target', 'action_y': 'action_target', 'timestamp_y': 'timestamp_target'}, inplace=True)
merged_test.rename(columns={'pin_x': 'pin_train', 'action_x': 'action_train', 'timestamp_x': 'timestamp_train', 'pin_y':'pin_target', 'action_y': 'action_target', 'timestamp_y': 'timestamp_target'}, inplace=True)

In [250]:
merged_train.head()

,user,pin_train,action_train,timestamp_train,pin_target,action_target,timestamp_target
0,1,"[239, 985, 999, 674, 994]","[1, 0, 1, 2, 2]","[2023-07-16 19:04:36.448844, 2023-07-17 19:04:...","[0, 0, 0, 0, 862]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 2023-07-15 19:04:36.448845]"
1,2,"[531, 163, 687, 778, 18]","[2, 2, 0, 1, 0]","[2023-07-16 19:04:36.453074, 2023-07-17 19:04:...","[0, 0, 352, 780, 664]","[0, 0, 0, 1, 0]","[0, 0, 2023-07-13 19:04:36.453078, 2023-07-14 ..."
2,4,"[630, 738, 798, 344, 885]","[2, 0, 2, 1, 2]","[2023-07-16 19:04:36.455611, 2023-07-17 19:04:...","[0, 0, 988, 978, 271]","[0, 0, 0, 2, 1]","[0, 0, 2023-07-13 19:04:36.455615, 2023-07-14 ..."
3,17,"[705, 442, 385, 103, 938]","[0, 1, 2, 2, 2]","[2023-07-16 19:04:36.469569, 2023-07-17 19:04:...","[0, 0, 0, 26, 510]","[0, 0, 0, 0, 1]","[0, 0, 0, 2023-07-14 19:04:36.469571, 2023-07-..."
4,20,"[558, 986, 672, 125, 354]","[1, 0, 2, 0, 2]","[2023-07-16 19:04:36.472624, 2023-07-17 19:04:...","[650, 123, 475, 865, 442]","[2, 0, 2, 1, 0]","[2023-07-11 19:04:36.472630, 2023-07-12 19:04:..."


In [251]:
train_dataset = RecommenderDataset(merged_train)
test_dataset = RecommenderDataset(merged_test)

train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [252]:
merged_test 

,user,pin_train,action_train,timestamp_train,pin_target,action_target,timestamp_target
0,12,"[925, 279, 516, 14, 422]","[1, 2, 1, 2, 1]","[2023-07-16 19:04:36.464736, 2023-07-17 19:04:...","[0, 0, 0, 345, 98]","[0, 0, 0, 1, 1]","[0, 0, 0, 2023-07-14 19:04:36.464738, 2023-07-..."
1,22,"[289, 872, 511, 798, 891]","[1, 1, 0, 0, 0]","[2023-07-16 19:04:36.475362, 2023-07-17 19:04:...","[448, 917, 861, 976, 888]","[0, 0, 1, 1, 2]","[2023-07-10 19:04:36.475369, 2023-07-11 19:04:..."
2,39,"[342, 942, 490, 141, 78]","[2, 2, 0, 0, 1]","[2023-07-16 19:04:36.492680, 2023-07-17 19:04:...","[0, 0, 210, 914, 945]","[0, 0, 2, 0, 0]","[0, 0, 2023-07-13 19:04:36.492684, 2023-07-14 ..."
3,45,"[493, 375, 755, 106, 973]","[1, 2, 2, 1, 0]","[2023-07-16 19:04:36.499065, 2023-07-17 19:04:...","[0, 0, 0, 133, 669]","[0, 0, 0, 2, 1]","[0, 0, 0, 2023-07-14 19:04:36.499067, 2023-07-..."
4,63,"[251, 553, 760, 127, 332]","[1, 1, 2, 0, 2]","[2023-07-16 19:04:36.517386, 2023-07-17 19:04:...","[0, 0, 0, 0, 157]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 2023-07-15 19:04:36.517387]"
5,70,"[742, 217, 562, 615, 211]","[2, 0, 1, 1, 0]","[2023-07-16 19:04:36.524893, 2023-07-17 19:04:...","[807, 473, 703, 28, 165]","[2, 0, 1, 2, 1]","[2023-07-10 19:04:36.524900, 2023-07-11 19:04:..."
6,82,"[675, 514, 671, 620, 222]","[1, 0, 0, 2, 1]","[2023-07-16 19:04:36.536892, 2023-07-17 19:04:...","[0, 0, 0, 486, 965]","[0, 0, 0, 1, 1]","[0, 0, 0, 2023-07-14 19:04:36.536895, 2023-07-..."
7,88,"[311, 22, 584, 95, 213]","[2, 1, 1, 0, 2]","[2023-07-16 19:04:36.543118, 2023-07-17 19:04:...","[0, 0, 369, 489, 883]","[0, 0, 1, 1, 1]","[0, 0, 2023-07-13 19:04:36.543122, 2023-07-14 ..."
8,89,"[167, 988, 599, 266, 999]","[0, 0, 1, 1, 2]","[2023-07-16 19:04:36.544120, 2023-07-17 19:04:...","[870, 826, 963, 164, 20]","[1, 1, 0, 0, 0]","[2023-07-11 19:04:36.544126, 2023-07-12 19:04:..."
9,94,"[16, 661, 831, 808, 529]","[0, 0, 0, 1, 0]","[2023-07-16 19:04:36.549058, 2023-07-17 19:04:...","[416, 947, 171, 677, 574]","[2, 2, 1, 2, 2]","[2023-07-10 19:04:36.549065, 2023-07-11 19:04:..."


In [237]:
#get embeddings for the first batch
#for batch in train_data_loader:
#
#    pin_embeddings = nn.Embedding(pins_vocab, pin_embedding_dim)
#    user, pin_ids, action_ids, target_pin_ids, target_action_ids = batch
#    #print(user)
#    #print(pin_ids.shape)
#    #print(action_ids.shape)
#    print(target_pin_ids.shape)
#    #print(target_pin_ids)
#
#    #user_embeddings = embedding_decoder(pin_ids,action_ids)
#    #print(user_embeddings.shape)
#
#    #for each user select the last embedding
#    #user_embeddings = user_embeddings[:,-1,:]
#    # Create a mask of non-zero elements
#    mask = target_pin_ids != 0 #shape [batch_size, seq_len]
#    # Get the indices of non-zero elements
#    indices = mask.nonzero() #shape [num_non_zero_elements, 2]
#
#    #print(mask)
#    #print(indices.shape)
#    #print(indices)  
#    # Get the unique row indices and the counts of non-zero elements in each row
#    unique_rows, counts = indices[:, 0].unique(return_counts=True) #shape [num_unique_rows, 1] #this is pulling the unique row indices and the counts of non-zero elements in each row
#    print(indices[:,0])
#    #print(unique_rows.size(0))
#    print(counts)
#    print(counts.cumsum(0)[:-1])
#    ## Get the starting indices of non-zero elements in each row in the 'indices' tensor
#    starts = torch.cat((torch.tensor([0]), counts.cumsum(0)[:-1]))
##
#    ## Generate a random number for each row in the range of the number of non-zero elements in the row
#    rand_num = torch.randint(counts.min(), size=(unique_rows.size(0),)).to(target_pin_ids.device)
#    print(rand_num)
#    ## Add the starting indices to the random numbers to get the indices of the selected elements in the 'indices' tensor
#    selected_indices = starts + rand_num
#    print(selected_indices)
#    ## Get the indices of the selected elements in the 'input_tensor'
#    selected_elements_indices = indices[selected_indices]
#    print(selected_elements_indices)
##
#    ## Get the selected elements
#    #selected_elements =target_pin_ids[selected_elements_indices[:, 0], selected_elements_indices[:, 1]]
#    #print(selected_elements.unsqueeze(1))
#
#    #for every user embedding multiply all the pin embeddings
#
#
#
#    
#    
#    break

In [236]:
#for batch in train_data_loader:
#
#    pin_embeddings = nn.Embedding(pins_vocab, pin_embedding_dim)
#    user, pin_ids, action_ids, target_pin_ids, target_action_ids = batch
#    #print(user)
#    #print(pin_ids.shape)
#    #print(action_ids.shape)
#    #print(target_pin_ids.shape)
#    #print(target_pin_ids)
#
#    #user_embeddings = embedding_decoder(pin_ids,action_ids)
#    #print(user_embeddings.shape)
#
#    #for each user select the last embedding
#    #user_embeddings = user_embeddings[:,-1,:]
#    # Create a mask of non-zero elements
#    mask = target_pin_ids != 0 #shape [batch_size, seq_len]
#    print(target_pin_ids)
#    non_zero_elements = target_pin_ids[mask]
#    print(non_zero_elements)
#    indices = torch.multinomial(mask.float(), 1)
#    indices[indices < 0] = 0
#    print(indices)
#    print(indices.shape)
#    #print(indices.squeeze())
#    print(indices.squeeze().shape)
#    #print(torch.arange(indices.size(0)))
#    # Reshape to 8x1
#    print(torch.arange(target_pin_ids.shape[0]))
#    random_numbers = target_pin_ids[torch.arange(target_pin_ids.shape[0]), indices.squeeze()]
#    random_numbers = random_numbers.view(-1, 1)
#
#    #print(random_numbers)   
#    #embedded = pin_embeddings(random_numbers)
#    #print(embedded.shape)
#    #print(embedded.transpose(2,1).shape)
#    # Get unique pins seen by user
#    #negative sampling for each user
#    # Find where the matrix has zeros
#    unique_pins = torch.unique(target_pin_ids, dim=1)
#
#       
#
#  
#
#    break

In [49]:
#class MyModel(nn.Module):
#    def __init__(self, pins_vocab, pin_embedding_dim):
#        super(MyModel, self).__init__()
#        self.pin_embeddings = nn.Embedding(pins_vocab, pin_embedding_dim)
#
#    def forward(self, pin_ids):
#        return self.pin_embeddings(pin_ids)
#
## Outside your loop
#model = MyModel(pins_vocab, pin_embedding_dim)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#
#for batch in train_data_loader:
#
#    user, pin_ids, action_ids, target_pin_ids, target_action_ids = batch
#
#    mask = target_pin_ids != 0 #shape [batch_size, seq_len]
#    indices = torch.multinomial(mask.float(), 1)
#    indices[indices < 0] = 0
#
#    random_numbers = target_pin_ids[torch.arange(target_pin_ids.shape[0]), indices.squeeze()]
#    random_numbers = random_numbers.view(-1, 1)
#
#    embedded = model(random_numbers)
#
#    dot_product = torch.bmm(embedded, embedded.transpose(2,1))
#
#    sigmoid = torch.sigmoid(dot_product)
#    #print(sigmoid.shape)
#    #squeeze the sigmoid
#    preds = sigmoid.squeeze(1)
#    #print(preds)
#    print(preds.shape)
#
#
#    negative_pin_embeddings = self.pin_embedding(torch.randint(0,  pin_embedding_shape, (8,)))
#        negative_pin_dot_product = torch.bmm(user_embeddings, negative_pin_embeddings.transpose(2,1))
#        negative_pin_sigmoid = torch.sigmoid(negative_pin_dot_product)
#
#        positiv_target = torch.ones_like(sigmoid)
#        negative_target = torch.zeros_like(negative_pin_sigmoid)
#
#
#
#    target = torch.ones_like(preds) # or torch.zeros_like(preds) if you want the model to output values close to 0
#    loss = F.binary_cross_entropy(preds, target)
#    print(loss)
#
#    optimizer.zero_grad() #zero out the gradients
#    loss.backward() #calculate the gradients
#    optimizer.step() #update the weights
#
#    break
#

In [93]:
#pin_embedding = nn.Embedding(pins_vocab, pin_embedding_dim)
#pin_embedding_shape = pin_embedding.weight.shape[0]
#print(pin_embedding_shape)
#
##sample negative pin embeddings for batch size 8
#negative_indices = torch.randint(0,  pin_embedding_shape, (8,))
#print(negative_indices)
#negative_pin_embeddings = pin_embedding(negative_indices)
#print(negative_pin_embeddings.shape)

In [239]:
#loading the pin_embeddings
import pickle
pin_embeddings = pickle.load(open("embeddings.pkl", "rb"))
pin_embeddings.shape

torch.Size([1000, 384])

In [240]:
import pytorch_lightning as pl
class RecommenderSystem(pl.LightningModule):

    def __init__(self, num_attention_heads,num_layers, pin_embeddings ,actions_vocab, hidden_size, pin_embedding_dim, action_embedding_dim, learning_rate=0.0001):
        super().__init__()
        self.user_embedding = TransferDecoder(num_attention_heads,num_layers, pin_embeddings,actions_vocab,action_embedding_dim, hidden_size)
        #convert the numpy array to a tensor and then to an embedding layer
        pin_embeddings = torch.tensor(pin_embeddings)
        self.pin_embedding = nn.Embedding.from_pretrained(pin_embeddings)
        self.pin_embedding.weight.requires_grad = False
        self.linear = nn.Linear(384, hidden_size)
        
    def forward(self, user ,pin_ids, action_ids,target_pin_ids):
        #print(f'user shape {user.shape[0]}')
         #positive pins sampled from the target pins
        mask = target_pin_ids != 0 #shape [batch_size, seq_len]
        indices = torch.multinomial(mask.float(), 1)
        indices[indices < 0] = 0
    
        random_numbers = target_pin_ids[torch.arange(target_pin_ids.shape[0]), indices.squeeze()]
        random_numbers = random_numbers.view(-1, 1)
    
        user_embeddings = self.user_embedding(pin_ids,action_ids)
        #print(f'  user_embeddings  {user_embeddings.shape}')
        #extract the last embedding for each user
        user_embeddings = user_embeddings[:,-1,:]
        #print(f'  user_embeddings  {user_embeddings.shape}')

        
        user_embeddings = user_embeddings.unsqueeze(1)
        print(f'  user_embeddings  {user_embeddings.shape}')
       

        positive_pin_embeddings = self.pin_embedding(random_numbers)
        #print(f'  positive_pin_embeddings  {positive_pin_embeddings.shape}')
        #positive_pin_embeddings = positive_pin_embeddings.squeeze(1)
        positive_pin_embedding  = self.linear(positive_pin_embeddings)
        print(f'  positive_pin_embeddings  {positive_pin_embedding.shape}')
    
        dot_product = torch.bmm(user_embeddings, positive_pin_embedding.transpose(2,1))
        dot_product = dot_product.squeeze(1)
        #print(f'  dot_product  {dot_product.shape}')
        sigmoid = torch.sigmoid(dot_product)
        #negative pins sampled randomly
        
        pin_embedding_shape = self.pin_embedding.weight.shape[0]
        #print(pin_embedding_shape)

        #sample negative pin embeddings for batch size 8
        negative_indices = torch.randint(0,  pin_embedding_shape, (user.shape[0],))
        #print(negative_indices)
        negative_pin_embeddings = self.pin_embedding(negative_indices)
        negative_pin_embeddings = self.linear(negative_pin_embeddings)
        
        negative_pin_embeddings = negative_pin_embeddings.unsqueeze(1)
        #print(f'  negative_pin_embeddings  {negative_pin_embeddings.shape}')
        #print(f'  user_embeddings  {user_embeddings.shape}')

        negative_pin_dot_product = torch.bmm(user_embeddings, negative_pin_embeddings.transpose(2,1))
        negative_pin_dot_product = negative_pin_dot_product.squeeze(1)
        negative_pin_sigmoid = torch.sigmoid(negative_pin_dot_product)

        total_sigmoid = torch.cat([sigmoid, negative_pin_sigmoid], dim=1)
        
        targets = torch.ones_like(sigmoid)
        targets = torch.cat([targets, torch.zeros_like(negative_pin_sigmoid)], dim=1)


        preds = total_sigmoid.squeeze(1)
        
        return preds, targets
        
    
    def training_step(self, batch):
        
        user = batch['user']
        pin_ids = batch['pin_ids']
        action_ids = batch['action_ids']
        target_pin_ids = batch['target_pin_ids']
        preds, targets  = self.forward(user,pin_ids, action_ids, target_pin_ids)
        #targets = torch.ones_like(preds)
        loss = F.binary_cross_entropy(preds, targets)
        self.log('train_loss', loss)
        return loss

    



    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer



model = RecommenderSystem(num_attention_heads,num_layers, pin_embeddings ,actions_vocab, hidden_size, 384, action_embedding_dim, learning_rate=0.0001)

trainer = pl.Trainer(max_epochs=10)

trainer.fit(model, train_data_loader, test_data_loader)





/var/folders/3z/c5w_1rg1087580hdbqj2wg0h0000gn/T/ipykernel_85402/260347766.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pin_embeddings = torch.tensor(pin_embeddings)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ankitkothari/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")

  | Name           | Type            | Params
---------------------------------------------------
0 | user_embedding | TransferDecoder | 1.3 M 
1 | pin_embedding  | Embedding       | 384 K 
2 | linear       

Training: 0it [00:00, ?it/s]

  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([7, 1, 12])
  positive_pin_embeddings  torch.Size([7, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([7, 1, 12])
  positive_pin_embeddings  torch.Size([7, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size([8, 1, 12])
  user_embeddings  torch.Size([8, 1, 12])
  positive_pin_embeddings  torch.Size(

`Trainer.fit` stopped: `max_epochs=10` reached.


  user_embeddings  torch.Size([7, 1, 12])
  positive_pin_embeddings  torch.Size([7, 1, 12])


In [253]:
import pytorch_lightning as pl
class RecommenderSystem(pl.LightningModule):

    def __init__(self, num_attention_heads,num_layers, pin_embeddings ,actions_vocab, hidden_size, pin_embedding_dim, action_embedding_dim, learning_rate=0.0001):
        super().__init__()
        self.user_embedding = TransferDecoder(num_attention_heads,num_layers, pin_embeddings,actions_vocab,action_embedding_dim, hidden_size)
        #convert the numpy array to a tensor and then to an embedding layer
        pin_embeddings = torch.tensor(pin_embeddings)
        self.pin_embedding = nn.Embedding.from_pretrained(pin_embeddings)
        self.pin_embedding.weight.requires_grad = False
        self.linear = nn.Linear(384, hidden_size)
        
    def forward(self, user_embeddings, pin_embeddings):
        dot_product = torch.bmm(user_embeddings, pin_embeddings.transpose(2,1))
        dot_product = dot_product.squeeze(1)
        sigmoid = torch.sigmoid(dot_product)
        preds = sigmoid.squeeze(1)
        return preds
        
    
    def training_step(self, batch):
        
        user = batch['user']
        pin_ids = batch['pin_ids']
        action_ids = batch['action_ids']
        target_pin_ids = batch['target_pin_ids']
        
        
        #user embeddings
        mask = target_pin_ids != 0 #shape [batch_size, seq_len]
        indices = torch.multinomial(mask.float(), 1)
        indices[indices < 0] = 0
        random_numbers = target_pin_ids[torch.arange(target_pin_ids.shape[0]), indices.squeeze()]
        random_numbers = random_numbers.view(-1, 1)
        user_embeddings = self.user_embedding(pin_ids,action_ids) #in: [batch_size, seq_len] out: mean_pooling [batch_size, seq_len, hidden_size]
        user_embeddings = user_embeddings[:,-1,:] #extract the last embedding for each user [batch_size, hidden_size]
        user_embeddings = user_embeddings.unsqueeze(1) #add a dimension for the sequence length [batch_size, 1, hidden_size]

        #positive pin embeddings
        positive_pin_embeddings = self.pin_embedding(random_numbers) #shape [batch_size, 1, hidden_size]
        positive_pin_embedding  = self.linear(positive_pin_embeddings) #shape [batch_size, 1, hidden_size]
        positive_preds = self.forward(user_embeddings, positive_pin_embedding) #shape [batch_size, 1]
        

        #negative pin embeddings
        pin_embedding_shape = self.pin_embedding.weight.shape[0] #shape [num_pins, hidden_size]
        negative_indices = torch.randint(0,  pin_embedding_shape, (user.shape[0],)) #shape [batch_size,]
        negative_pin_embeddings = self.pin_embedding(negative_indices) #shape [batch_size, hidden_size]
        negative_pin_embeddings = self.linear(negative_pin_embeddings) #shape [batch_size, hidden_size]
        negative_pin_embeddings = negative_pin_embeddings.unsqueeze(1) #shape [batch_size, 1, hidden_size]
        negative_pin_preds = self.forward(user_embeddings, negative_pin_embeddings) #shape [batch_size, 1]
        
        #concatenate the positive and negative pin preds
        preds = torch.cat([positive_preds, negative_pin_preds]) #shape [batch_size*2, 1]
        targets = torch.ones_like(positive_preds) #shape [batch_size, 1]
        targets = torch.cat([targets, torch.zeros_like(negative_pin_preds)]) #shape [batch_size*2, 1]
         #shape [batch_size*2, 1]

        #calculate the loss
        loss = F.binary_cross_entropy(preds, targets)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        user = batch['user']
        pin_ids = batch['pin_ids']
        action_ids = batch['action_ids']
        target_pin_ids = batch['target_pin_ids']

        #user embeddings
        mask = target_pin_ids != 0 #shape [batch_size, seq_len]
        indices = torch.multinomial(mask.float(), 1)
        indices[indices < 0] = 0
        random_numbers = target_pin_ids[torch.arange(target_pin_ids.shape[0]), indices.squeeze()]
        random_numbers = random_numbers.view(-1, 1)
        user_embeddings = self.user_embedding(pin_ids,action_ids) #in: [batch_size, seq_len] out: mean_pooling [batch_size, seq_len, hidden_size]
        user_embeddings = user_embeddings[:,-1,:] #extract the last embedding for each user [batch_size, hidden_size]
        user_embeddings = user_embeddings.unsqueeze(1) #add a dimension for the sequence length [batch_size, 1, hidden_size]

        #positive pin embeddings
        positive_pin_embeddings = self.pin_embedding(random_numbers) #shape [batch_size, 1, hidden_size]
        positive_pin_embedding  = self.linear(positive_pin_embeddings) #shape [batch_size, 1, hidden_size]
        positive_preds = self.forward(user_embeddings, positive_pin_embedding) #shape [batch_size, 1]

        #negative pin embeddings
        pin_embedding_shape = self.pin_embedding.weight.shape[0] #shape [num_pins, hidden_size]
        negative_indices = torch.randint(0,  pin_embedding_shape, (user.shape[0],)) #shape [batch_size,]
        negative_pin_embeddings = self.pin_embedding(negative_indices) #shape [batch_size, hidden_size]
        negative_pin_embeddings = self.linear(negative_pin_embeddings) #shape [batch_size, hidden_size]
        negative_pin_embeddings = negative_pin_embeddings.unsqueeze(1) #shape [batch_size, 1, hidden_size]
        negative_pin_preds = self.forward(user_embeddings, negative_pin_embeddings) #shape [batch_size, 1]

        #concatenate the positive and negative pin preds
        preds = torch.cat([positive_preds, negative_pin_preds]) #shape [batch_size*2, 1]
        targets = torch.ones_like(positive_preds) #shape [batch_size, 1]
        targets = torch.cat([targets, torch.zeros_like(negative_pin_preds)]) #shape [batch_size*2, 1]

        #calculate the loss
        loss = F.binary_cross_entropy(preds, targets)
        self.log('val_loss', loss)
        return loss


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer



model = RecommenderSystem(num_attention_heads,num_layers, pin_embeddings ,actions_vocab, hidden_size, 384, action_embedding_dim, learning_rate=0.0001)

trainer = pl.Trainer(max_epochs=10)

trainer.fit(model, train_data_loader, test_data_loader)

/var/folders/3z/c5w_1rg1087580hdbqj2wg0h0000gn/T/ipykernel_85402/3111819979.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pin_embeddings = torch.tensor(pin_embeddings)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type            | Params
---------------------------------------------------
0 | user_embedding | TransferDecoder | 1.3 M 
1 | pin_embedding  | Embedding       | 384 K 
2 | linear         | Linear          | 4.6 K 
---------------------------------------------------
1.3 M     Trainable params
384 K     Non-trainable params
1.7 M     Total params
6.807     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/ankitkothari/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ankitkothari/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ankitkothari/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (4) is smaller th

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [254]:
#save the model
torch.save(model.state_dict(), 'model.pt')

#load the model
model = RecommenderSystem(num_attention_heads,num_layers, pin_embeddings ,actions_vocab, hidden_size, 384, action_embedding_dim, learning_rate=0.0001)
model.load_state_dict(torch.load('model.pt'))

/var/folders/3z/c5w_1rg1087580hdbqj2wg0h0000gn/T/ipykernel_85402/3111819979.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pin_embeddings = torch.tensor(pin_embeddings)


<All keys matched successfully>

In [255]:
#provide a user id and get the top 10 recommendations
#random new user with 5 pins randomly selected

#sample a new user
new_user = torch.randint(0,  pins_vocab, (1,5))
new_user_actions = torch.randint(0,  actions_vocab, (1,5))
print(new_user)

tensor([[681, 155, 733,  62, 727]])


In [256]:
#make top 10 recommendations
model.eval()
with torch.no_grad():
    user_embeddings = model.user_embedding(new_user,new_user_actions)
    user_embeddings = user_embeddings[:,-1,:]
    user_embeddings = user_embeddings.unsqueeze(1)
    
    #get the pin embeddings
    pin_embeddings = model.pin_embedding.weight
    pin_embeddings = model.linear(pin_embeddings)
    pin_embeddings = pin_embeddings.unsqueeze(0)
    print(pin_embeddings.shape)

    dot_product = torch.bmm(user_embeddings, pin_embeddings.transpose(2,1))
    dot_product = dot_product.squeeze(1)

    sigmoid = torch.sigmoid(dot_product)
    #print(sigmoid.shape)
    #squeeze the sigmoid
    preds = sigmoid.squeeze(1)
    #print(preds)
    print(preds.shape)

    #get the top 10 recommendations
    top_10 = torch.topk(preds, 10)

    #get the indices of the top 10 recommendations
    top_10_indices = top_10.indices
    print(top_10_indices)

torch.Size([1, 1000, 12])
torch.Size([1, 1000])
tensor([[970, 229,  44, 641, 194, 483, 397, 331, 583,  16]])
